Primeras palabras

In [ ]:
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()
API_KEY = os.getenv("AZURE_OPENAI_KEY")
ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
DEPLOYMENT_GPT = os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT")
API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=ENDPOINT
)

df = pd.read_excel("df_ECE.xlsx")
texto_a_corregir = df.iloc[0]["escrito"]

prompt_ortografia = """
Eres un asistente experto en corrección de textos. Aplica estas reglas al texto:

1. ORTOGRAFÍA
   1.1 Ortografía acentual: errores en tildes según normas generales y especiales.
       Ejemplos: esta/está, animos/ánimos
   1.2 Ortografía literal: errores en letras dentro de palabras.
       Omisión: "alchol" → "alcohol"
       Sustitución: "fotocopeo" → "fotocopio"
       Adición: "rrallado" → "rallado"
       Ejemplos adicionales: "apartir" → "a partir", "através" → "a través", "porsiacaso" → "por si acaso"
   1.3 Ortografía puntual: errores en signos de puntuación según normas RAE.
       Incluye: coma, punto, punto y coma, signos de exclamación e interrogación

INSTRUCCIONES CRÍTICAS:
- Detecta errores según estas categorías y subcategorías.
- NO incluyas el texto corregido completo.
- Devuelve SOLAMENTE JSON válido sin formato markdown (sin ```json o similares).
- El JSON debe comenzar inmediatamente con {{ y terminar con }}.

ESTRUCTURA EXACTA REQUERIDA:
{
  "texto_original_preview": "Primeros 100 caracteres del texto...",
  "errores_detectados": {
      "ortografia": ["error → corrección"]
  }
}

- En "texto_original_preview" incluye SOLO los primeros 100 caracteres del texto original.
- En "errores_detectados" lista todos los errores encontrados.
- NO uses markdown, NO incluyas ```json, devuelve SOLO el JSON.
"""

messages = [
    {"role": "system", "content": prompt_ortografia},
    {"role": "user", "content": texto_a_corregir}
]

try:
    response = client.chat.completions.create(
        model=DEPLOYMENT_GPT,
        messages=messages,
        temperature=0,
        response_format={"type": "json_object"} 
    )

    respuesta_limpia = response.choices[0].message.content.strip()
    resultado_json = json.loads(respuesta_limpia)

    # --- Guardado ---
    # output_path = "resultado_corr.json"
    # with open(output_path, "w", encoding="utf-8") as f:
    #     json.dump(resultado_json, f, ensure_ascii=False, indent=4)
    # print(f"Resultado guardado en {output_path}")

    # Imprimir preview de solo 20 caracteres
    preview = resultado_json.get("texto_original_preview", "")[:20]
    print("Preview del texto (20 chars):", preview)

    # Imprimir errores detectados
    errores = resultado_json.get("errores_detectados", {})
    print("Errores detectados:", json.dumps(errores, ensure_ascii=False, indent=2))

except json.JSONDecodeError as e:
    print(f"Error al parsear JSON: {e}")
    print("Respuesta cruda recibida:")
    print(respuesta_limpia)

    with open("respuesta_crudar.txt", "w", encoding="utf-8") as f:
        f.write(respuesta_limpia)

Preview del texto (20 chars): Los animos en nuestr
Errores detectados: {
  "ortografia": [
    "animos → ánimos",
    "esta → está",
    "conversacion → conversación",
    "mas → más",
    "ultima → última",
    "claustrofobicas → claustrofóbicas",
    "porque → por qué",
    "quien → quién",
    "velara → velará",
    "deberia → debería",
    "pretenciones → pretensiones",
    "regulacion → regulación",
    "ejecucion → ejecución",
    "practica → práctica",
    "limitarsele → limitarse le",
    "vuestra → nuestra",
    "asi → así",
    "requieres en material → requieres el material",
    "desentenderse del claro → desentenderse claramente del",
    "psicologica → psicológica",
    "ultimos → últimos",
    "oportunides → oportunidades",
    "acabara → acabará",
    "eficiente → eficientemente",
    "vitalmente → vitalmente.",
    "disposicion → disposición",
    "pongamonos → pongámonos",
    "pasaria → pasaría",
    "malversacion → malversación",
    "malinterpretacion → malinterpret

In [6]:
df = pd.read_excel("df_ECE.xlsx")
texto_a_corregir = df.iloc[0]["escrito"]

prompt_vocabulario = """
Analiza el siguiente texto en base a la dimensión de VOCABULARIO:

2. VOCABULARIO
   2.1 Amplitud: evitar repetición innecesaria de sustantivos, adjetivos, verbos y adverbios.
   2.2 Precisión léxica: uso incorrecto de palabras que confunden el sentido; incluye redundancias.
   2.3 Registro formal: evitar expresiones informales, abreviaciones o coloquialismos.

Ejemplos:
- Amplitud: contaminación/ ecosistema / polución / ambiente
- Precisión léxica: “absceso” → “acceso”, “actitud” → “aptitud”; “salir para afuera”, “hace un tiempo atrás”
- Registro formal: “profe” → “profesor/a”, “finde” → “fin de semana”; “cachar” → “comprender”

INSTRUCCIONES CRÍTICAS:
- Detecta errores según estas categorías y subcategorías.
- NO incluyas el texto corregido completo.
- Devuelve SOLAMENTE JSON válido sin formato markdown (sin ```json o similares).
- El JSON debe comenzar inmediatamente con { y terminar con }.

ESTRUCTURA EXACTA REQUERIDA:
{
  "texto_original_preview": "Primeros 20 caracteres del texto...",
  "errores_detectados": {
      "vocabulario": ["error → corrección"]
  }
}

"""

messages = [
    {"role": "system", "content": prompt_vocabulario},
    {"role": "user", "content": texto_a_corregir}
]

try:
    response = client.chat.completions.create(
        model=DEPLOYMENT_GPT,
        messages=messages,
        temperature=0,
        response_format={"type": "json_object"} 
    )

    respuesta_limpia = response.choices[0].message.content.strip()
    resultado_json = json.loads(respuesta_limpia)

    # --- Guardar resultados en archivo JSON (opcional, descomentarlo si lo quieres usar) ---
    # output_path = "resultado_vocabulario.json"
    # with open(output_path, "w", encoding="utf-8") as f:
    #     json.dump(resultado_json, f, ensure_ascii=False, indent=4)
    # print(f"Resultado guardado en {output_path}")

    print("Preview texto original:", resultado_json.get("texto_original", "")[:20])
    print("Resultados análisis vocabulario:", resultado_json.get("resultados", ""))

except json.JSONDecodeError as e:
    print(f"Error al parsear JSON: {e}")
    print("Respuesta cruda recibida:")
    print(respuesta_limpia)



Preview texto original: 
Resultados análisis vocabulario: 


In [7]:
df = pd.read_excel("df_ECE.xlsx")
texto_a_corregir = df.iloc[0]["escrito"]

prompt_cohesion = """
Analiza el siguiente texto en base a la dimensión de COHESIÓN TEXTUAL:

3. COHESIÓN TEXTUAL
   3.1 Concordancia gramatical: errores de concordancia entre sustantivo/adjetivo, sujeto/predicado, pronombres y consistencia de tiempos verbales.
   3.2 Conexión textual: uso adecuado de preposiciones, locuciones y conectores; evitar queísmo/dequeísmo.
   3.3 Referencias y correferencias: pronombres u otros mecanismos sin referencia clara.
       Ejemplos: “esto”, “lo anterior” sin antecedente claro

INSTRUCCIONES CRÍTICAS:
- Detecta errores según estas categorías y subcategorías.
- NO incluyas el texto corregido completo.
- Devuelve SOLAMENTE JSON válido sin formato markdown (sin ```json o similares).
- El JSON debe comenzar inmediatamente con { y terminar con }.
"""

messages = [
    {"role": "system", "content": prompt_cohesion},
    {"role": "user", "content": texto_a_corregir}
]

try:
    response = client.chat.completions.create(
        model=DEPLOYMENT_GPT,
        messages=messages,
        temperature=0,
        response_format={"type": "json_object"} 
    )

    respuesta_limpia = response.choices[0].message.content.strip()
    resultado_json = json.loads(respuesta_limpia)

    # --- Guardar resultados en archivo JSON (opcional, descomentar si se quiere usar) ---
    # output_path = "resultado_cohesion.json"
    # with open(output_path, "w", encoding="utf-8") as f:
    #     json.dump(resultado_json, f, ensure_ascii=False, indent=4)
    # print(f"Resultado guardado en {output_path}")

    print("Preview texto original:", resultado_json.get("texto_original", "")[:20])
    print("Resultados análisis cohesión:", resultado_json.get("resultados", ""))

except json.JSONDecodeError as e:
    print(f"Error al parsear JSON: {e}")
    print("Respuesta cruda recibida:")
    print(respuesta_limpia)


Preview texto original: 
Resultados análisis cohesión: 


In [11]:
prompt_completo = """
Eres un asistente experto en corrección de textos. Aplica estas reglas al texto:

CATEGORÍAS Y SUBCATEGORÍAS DE ERRORES:

1. ORTOGRAFÍA
   1.1 Ortografía acentual: errores en tildes según normas generales y especiales.
       Ejemplos: esta/está, animos/ánimos
   1.2 Ortografía literal: errores en letras dentro de palabras.
       Omisión: "alchol" → "alcohol"
       Sustitución: "fotocopeo" → "fotocopio"
       Adición: "rrallado" → "rallado"
       Ejemplos adicionales: “apartir” → “a partir”, “através” → “a través”, “porsiacaso” → “por si acaso”
   1.3 Ortografía puntual: errores en signos de puntuación según normas RAE.
       Incluye: coma, punto, punto y coma, signos de exclamación e interrogación

2. VOCABULARIO
   2.1 Amplitud: evitar repetición innecesaria de sustantivos, adjetivos, verbos y adverbios.
       Ejemplo: contaminación/ ecosistema / polución / ambiente
   2.2 Precisión léxica: uso incorrecto de palabras que confunden el sentido; incluye redundancias.
       Ejemplos: “absceso” → “acceso”, “actitud” → “aptitud”; “salir para afuera”, “hace un tiempo atrás”
   2.3 Registro formal: evitar expresiones informales, abreviaciones o coloquialismos.
       Ejemplos: “profe” → “profesor/a”, “finde” → “fin de semana”; “cachar” → “comprender”

3. COHESIÓN TEXTUAL
   3.1 Concordancia gramatical: errores de concordancia entre sustantivo/adjetivo, sujeto/predicado, pronombres y consistencia de tiempos verbales.
   3.2 Conexión textual: uso adecuado de preposiciones, locuciones y conectores; evitar queísmo/dequeísmo.
   3.3 Referencias y correferencias: pronombres u otros mecanismos sin referencia clara.
       Ejemplos: “esto”, “lo anterior” sin antecedente claro

INSTRUCCIONES CRÍTICAS:
- Detecta errores según estas categorías y subcategorías.
- Para CADA error, especifica la subcategoría exacta.
- NO incluyas el texto corregido completo.
- Devuelve SOLAMENTE JSON válido sin formato markdown.
- El JSON debe comenzar inmediatamente con {{ y terminar con }}.

ESTRUCTURA EXACTA REQUERIDA:
{{
  "texto_original_preview": "Primeros 100 caracteres del texto...",
  "errores_detectados": {{
      "ortografia": [
        {{"error": "error original", "correccion": "corrección propuesta", "subcategoria": "acentual/literal/puntual"}}
      ],
      "vocabulario": [
        {{"error": "error original", "correccion": "corrección propuesta", "subcategoria": "amplitud/precision_lexica/registro_formal"}}
      ],
      "cohesion": [
        {{"error": "error original", "correccion": "corrección propuesta", "subcategoria": "concordancia/conexion/referencias"}}
      ]
  }}
}}
"""

# Enviar la petición a Azure OpenAI
messages = [
    {"role": "system", "content": prompt_completo},
    {"role": "user", "content": texto_a_corregir}
]

try:
    response = client.chat.completions.create(
        model=DEPLOYMENT_GPT,
        messages=messages,
        temperature=0,
        response_format={"type": "json_object"}
    )

    respuesta_limpia = response.choices[0].message.content.strip()
    resultado_json = json.loads(respuesta_limpia)

    # Filtrar solo subcategorías válidas
    categorias_validas = {
        "ortografia": {"acentual", "literal", "puntual"},
        "vocabulario": {"amplitud", "precision_lexica", "registro_formal"},
        "cohesion": {"concordancia_gramatical", "conexion_textual", "referencias_y_correferencias"}
    }

    for categoria, subcategorias in resultado_json["errores_detectados"].items():
        resultado_json["errores_detectados"][categoria] = [
            error for error in subcategorias if error["subcategoria"] in categorias_validas[categoria]
        ]

    # --- Guardar en archivo JSON ---
    output_path = "resultado_correccion.json"
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(resultado_json, f, ensure_ascii=False, indent=4)

    print(f"Resultado guardado en {output_path}")
    print("Preview del texto:", resultado_json.get("texto_original_preview", ""))
    
    # Estadísticas de errores
    if "errores_detectados" in resultado_json:
        errores = resultado_json["errores_detectados"]
        print("\nEstadísticas de errores:")
        for categoria, lista_errores in errores.items():
            if lista_errores:
                subcategorias = {}
                for error in lista_errores:
                    subcat = error.get("subcategoria", "sin_subcategoria")
                    subcategorias[subcat] = subcategorias.get(subcat, 0) + 1
                
                print(f"{categoria.upper()}: {len(lista_errores)} errores")
                for subcat, count in subcategorias.items():
                    print(f"  - {subcat}: {count}")

except json.JSONDecodeError as e:
    print(f"Error al parsear JSON: {e}")
    print("Respuesta cruda recibida:")
    print(respuesta_limpia)


Resultado guardado en resultado_correccion.json
Preview del texto: Los animos en nuestra actual pandemia no se hallan errados, la salud mental con respecto a lo que podemos denominar como la realidad del trabajo permanente no esta enfrentando nada nuevo...

Estadísticas de errores:
ORTOGRAFIA: 14 errores
  - acentual: 9
  - literal: 5
VOCABULARIO: 6 errores
  - precision_lexica: 3
  - registro_formal: 3
